In [76]:
import pandas as pd
import geopandas as gpd

In [77]:
# only keep cycleways

In [91]:
run = 8
agg = "segments"

In [92]:
gdf = gpd.read_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_{agg}_pred.shp', dtype={'ID': int}, crs=25833)

In [80]:
# use predefined road type as target
gdf['target_road_type'] = gdf['MAß_SEITE'].apply(lambda x: 'cycleway' if x == 'Beidseitig'
                                                 else ('road' if x in ('Gesamte Straße', 'Gesamte Straß') 
                                                       else None))

In [81]:
gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE'].apply(lambda x: 'cycleway' if x in ('bike_lane', 'cycleway') else x)

In [83]:
gdf.target_road_type.value_counts()

target_road_type
cycleway    894
road        561
Name: count, dtype: int64

In [84]:
# only keep ways where target_road_type and road_type match
gdf = gdf[(gdf['target_road_type'] == gdf['ROAD_TYPE_refined'])| (gdf['target_road_type'].isnull())].copy()

In [85]:
# how to choose bike lane or cycleway? first: which has more segments covered, second, which has more images

In [86]:
road_type_order = ['cycleway', 'road', 'pedestrian', 'other']

# select one road type per way
if agg == "ways":

    # for each road type, take the classification with the most votes (and segments if votes are tied)
    gdf.sort_values(['ID', 'SEGMENT_VO', 'AVG_IMG_CO'], ascending=False, inplace=True)
    gdf = gdf.groupby(['ID', 'ROAD_TYPE_refined']).first().reset_index()


    # Convert the ROAD_TYPE column to a categorical type with the specified order: use cycleways first
    gdf['ROAD_TYPE_refined'] = pd.Categorical(gdf['ROAD_TYPE_refined'], categories=road_type_order, ordered=True)
    gdf.sort_values(['ID', 'ROAD_TYPE_refined'], inplace=True)
    # convert back to string to save as gpkg
    gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE_refined'].astype(str)
    gdf = gdf.groupby('ID').first().reset_index()

elif agg == "segments":
    gdf.sort_values(['ID', 'GROUP_NUM','SEGMENT_VO', 'AVG_IMG_CO'], ascending=False, inplace=True)
    gdf = gdf.groupby(['ID','GROUP_NUM', 'ROAD_TYPE_refined']).first().reset_index()

    # Convert the ROAD_TYPE column to a categorical type with the specified order: use cycleways first
    gdf['ROAD_TYPE_refined'] = pd.Categorical(gdf['ROAD_TYPE_refined'], categories=road_type_order, ordered=True)
    gdf.sort_values(['ID', 'GROUP_NUM', 'ROAD_TYPE_refined'], inplace=True)
    # convert back to string to save as gpkg
    gdf['ROAD_TYPE_refined'] = gdf['ROAD_TYPE_refined'].astype(str)
    gdf = gdf.groupby(['ID','GROUP_NUM']).first().reset_index()


In [88]:
gdf['ID'] = gdf['ID'].astype(int)

In [90]:
gdf.to_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/berlin_{agg}_pred_cycleways.shp')

/var/folders/m7/jb7xk8h91p541g43zv2r8l740000gn/T/ipykernel_38885/2451296086.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(f'/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run{run}/{file}.shp')
